In [5]:
import pandas as pd
import nltk
import collections
import spacy
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nlp = spacy.load('en')
from fuzzywuzzy import fuzz
import textblob
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from scipy import spatial
from bs4 import BeautifulSoup
import dill  

In [6]:
df = pd.read_csv("/home/utkarsh/Desktop/train.csv")

with open("/home/utkarsh/Desktop/train.csv") as f:
    neg_dict=f.readlines()
neg_dict=[s.replace('\n', '') for s in neg_dict]
qids = pd.Series(df['qid1'].tolist() + df['qid2'].tolist())
unique_qs = len(np.unique(qids))
qs_morethan_onetime = np.sum(qids.value_counts() > 1)
q_vals=qids.value_counts()
q_vals=q_vals.values
df = df.fillna('')
df['freq_qid1'] = df.groupby('qid1')['qid1'].transform('count') 
df['freq_qid2'] = df.groupby('qid2')['qid2'].transform('count')
df['q1len'] = df['question1'].str.len() 
df['q2len'] = df['question2'].str.len()
df['q1_n_words'] = df['question1'].apply(lambda row: len(row.split(" ")))
df['q2_n_words'] = df['question2'].apply(lambda row: len(row.split(" ")))

def abs_len(s1,s2):
    tokens1, tokens2=s1.split(), s2.split()
    return abs(len(tokens1)-len(tokens2))

def edit_dist(s1,s2):
    return nltk.edit_distance(s1,s2)

def jac_dist(s1,s2):
    return nltk.jaccard_distance(set(s1),set(s2))

def en_overlap(s1,s2):
    doc=nlp(s1)
    doc1=nlp(s2)
    if len(doc.ents)==0 and len(doc.ents)==0:
        return 0
    else:
        return nltk.jaccard_distance(set(doc.ents),set(doc1.ents))
        
def fuzz_ratio(s1,s2):
    return fuzz.ratio(s1,s2)

def doc_similarity(s1,s2):
    doc=nlp(s1)
    doc1=nlp(s2)
    return doc.similarity(doc1)


In [7]:
postag_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

def postag_count(s1,s2,flag):
    count=0
    count1=0
    try:
        tag=textblob.TextBlob(s1)
        tag1=textblob.TextBlob(s2)
        for t in tag.tags:
            p=list(t)[1]
            if p in postag_family[flag]:
                count += 1
        for t in tag1.tags:
            p=list(t)[1]
            if p in postag_family[flag]:
                count1 += 1
    except:
        pass
    return abs(count1-count)
def normalized_word_Common(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return 1.0 * len(w1 & w2)

def normalized_word_Total(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return 1.0 * (len(w1) + len(w2))


def normalized_word_share(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return 1.0 * len(w1 & w2)/(len(w1) + len(w2))

In [1]:

STOP_WORDS = stopwords.words("english")
def preprocess(x):
    x = str(x).lower()
    x = x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
                           .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will")
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    x = re.sub(r"([0-9]+)000", r"\1k", x)
    
    
    porter = PorterStemmer()
    pattern = re.compile('\W')
    
    if type(x) == type(''):
        x = re.sub(pattern, ' ', x)
    
    
    if type(x) == type(''):
        x = porter.stem(x)
        example1 = BeautifulSoup(x)
        x = example1.get_text()
               
    
    return x

df['word_Common'] = df.apply(normalized_word_Common, axis=1)
df['word_Total'] = df.apply(normalized_word_Total, axis=1)
df['word_share'] = df.apply(normalized_word_share, axis=1)
df['freq_q1+q2'] = df['freq_qid1']+df['freq_qid2']
df['freq_q1-q2'] = abs(df['freq_qid1']-df['freq_qid2'])
df['question1']=df['question1'].apply(lambda x:preprocess(x))
df['question2']=df['question2'].apply(lambda x:preprocess(x))
df['LCS'] = df.apply(lambda x: lcs(x['question1'], x['question2']), axis=1)
df['ABS_LEN']=df.apply(lambda x:abs_len(x['question1'],x['question2']),axis=1)
df['EDIT_DIST']=df.apply(lambda x:edit_dist(x['question1'],x['question2']),axis=1)
df['JAC_DIST']=df.apply(lambda x:jac_dist(x['question1'],x['question2']),axis=1)
df['EN_OVERLAP']=df.apply(lambda x:en_overlap(x['question1'],x['question2']),axis=1)
df['FUZZ_RATIO']=df.apply(lambda x:fuzz_ratio(x['question1'],x['question2']),axis=1)
df['DOC_SIMILARITY']=df.apply(lambda x:doc_similarity(x['question1'],x['question2']),axis=1)
df['NOUN_COUNT'] = df.apply(lambda x: postag_count(x['question1'],x['question2'],'noun'),axis=1)
df['VERB_COUNT'] = df.apply(lambda x: postag_count(x['question1'],x['question2'],'verb'),axis=1)
df['ADJ_COUNT'] = df.apply(lambda x: postag_count(x['question1'],x['question2'],'adj'),axis=1)
df['ADV_COUNT'] = df.apply(lambda x: postag_count(x['question'],x['qustion2'],'adv'),axis=1)
df['PRON_COUNT'] = df.apply(lambda x: postag_count(x['question1'],x['question2'],'pron'),axis=1)
df["token_set_ratio"]= df.apply(lambda x: fuzz.token_set_ratio(x["question1"], x["question2"]), axis=1)
df["token_sort_ratio"]= df.apply(lambda x: fuzz.token_sort_ratio(x["question1"], x["question2"]), axis=1)
df["fuzz_ratio"]= df.apply(lambda x: fuzz.QRatio(x["question1"], x["question2"]), axis=1)
df["fuzz_partial_ratio"]= df.apply(lambda x: fuzz.partial_ratio(x["question1"], x["question2"]), axis=1)
quora_dup_pred.py


NameError: name 'stopwords' is not defined